# Finding Games

To find a single game, use the `LeagueGameFinder` class.
While you can call it without any arguments and get ~30,000 games returned (I believe that's the max number of rows that nba.com will send in a response) across the NBA, WNBA, G-League, and international ball, it's a better idea to pass a team ID.

See `nba_api.stats.static.teams` in the [Basics Notebook](Basics.ipynb) for getting a team ID.

Let's try to find the last time the Celtics played the Raptors in 2017-18.
That will be four steps:
1. Fetch all Celtics games.
2. Select just games from the 2017-18 season (SEASON_ID ending in 2017).
3. Select games where the opponent is the Raptors (MATCHUP contains 'TOR').
4. Order by date and select the last row.

### Get All Celtics Games

In [1]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [2]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22018,1610612738,BOS,Boston Celtics,0021800145,2018-11-05,BOS @ DEN,L,240,107,...,0.833,7,33,40,22,9,9,12,21,-8.0
1,22018,1610612738,BOS,Boston Celtics,0021800127,2018-11-03,BOS @ IND,L,240,101,...,0.667,10,35,45,23,6,8,14,20,-1.0
2,22018,1610612738,BOS,Boston Celtics,0021800115,2018-11-01,BOS vs. MIL,W,240,117,...,0.846,8,34,42,30,5,8,12,23,4.0
3,22018,1610612738,BOS,Boston Celtics,0021800099,2018-10-30,BOS vs. DET,W,239,108,...,0.781,8,38,46,25,8,4,14,20,3.0
4,22018,1610612738,BOS,Boston Celtics,0021800074,2018-10-27,BOS @ DET,W,238,109,...,0.680,8,46,54,25,4,8,13,26,20.0


As you can see above, the season ID is 5 digits.
I believe the last 4 will always be the current season (2018 for the 2018-19 season).
We can do a sanity check and look at how many games the Celtics have played in recent years.

In [3]:
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]

,GAME_ID
SEASON_ID,
2015,103
2016,115
2017,113
2018,20


Note that some of these games are preseason and summer league, so these numbers aren't just regular season and playoffs.

### Filter to Games in the 2017-18 Season

In [4]:
games_1718 = games[games.SEASON_ID.str[-4:] == '2017']
games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
20,42017,1610612738,BOS,Boston Celtics,0041700307,2018-05-27,BOS vs. CLE,L,240,79,...,0.737,11,31,42,18,6,0,5,20,-8.0
21,42017,1610612738,BOS,Boston Celtics,0041700306,2018-05-25,BOS @ CLE,L,241,99,...,0.550,5,26,31,25,5,2,13,18,-10.0
22,42017,1610612738,BOS,Boston Celtics,0041700305,2018-05-23,BOS vs. CLE,W,240,96,...,0.913,7,38,45,18,10,6,8,19,13.0
23,42017,1610612738,BOS,Boston Celtics,0041700304,2018-05-21,BOS @ CLE,L,239,102,...,0.767,9,28,37,21,9,3,9,26,-9.0
24,42017,1610612738,BOS,Boston Celtics,0041700303,2018-05-19,BOS @ CLE,L,240,86,...,0.786,6,28,34,16,4,4,15,25,-30.0


### Filter to Games Against the Raptors

In [5]:
raps_games_1718 = games_1718[games_1718.MATCHUP.str.contains('TOR')]
raps_games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
43,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,...,0.893,8,35,43,10,7,3,16,16,-18.0
45,22017,1610612738,BOS,Boston Celtics,0021701140,2018-03-31,BOS vs. TOR,W,240,110,...,0.852,8,33,41,19,6,3,5,23,11.0
66,22017,1610612738,BOS,Boston Celtics,0021700798,2018-02-06,BOS @ TOR,L,240,91,...,0.750,9,29,38,21,3,3,17,19,-20.0
107,22017,1610612738,BOS,Boston Celtics,0021700188,2017-11-12,BOS vs. TOR,W,239,95,...,0.789,15,31,46,24,9,2,14,18,1.0


### Sort by Game Date and Select the Last Row

In [6]:
last_raps_game = raps_games_1718.sort_values('GAME_DATE').iloc[-1]
last_raps_game

SEASON_ID                     22017
TEAM_ID                  1610612738
TEAM_ABBREVIATION               BOS
TEAM_NAME            Boston Celtics
GAME_ID                  0021701171
GAME_DATE                2018-04-04
MATCHUP                   BOS @ TOR
WL                                L
MIN                             237
PTS                              78
FGM                              25
FGA                              75
FG_PCT                        0.333
FG3M                              3
FG3A                             22
FG3_PCT                       0.136
FTM                              25
FTA                              28
FT_PCT                        0.893
OREB                              8
DREB                             35
REB                              43
AST                              10
STL                               7
BLK                               3
TOV                              16
PF                               16
PLUS_MINUS                  

There it is.

We can see the game was on April 4th, was in Toronto, and ended in an 18-point Raptors victory.
It can be confusing to read this, but the row is all relative to the Celtics (the team we queried).
All the stats (points, rebounds, blocks, plus/minus) are theirs.

If we wanted stats for both teams (a common use case), we'd need to get *both* rows for this game ID.
There are always two, one with stats for each team.
So back to the `LeagueGameFinder`!

In [7]:
game_id = last_raps_game.GAME_ID
game_id

'0021701171'

In [35]:
# Get **all** the games so we can filter to an individual GAME_ID
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]
# Find the game_id we want
full_game = all_games[all_games.GAME_ID == game_id]
full_game

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1535,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,...,0.750,12,36,48,23,10,6,10,25,18.0
1537,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,...,0.893,8,35,43,10,7,3,16,16,-18.0


Two rows, one with the Celtics' stats and one with the Raptors'.
You may want to join these these two rows into one, so you have stats for both teams in the same observation.

This function will work for larger datasets too (even though we just have one game here);
you can run it on any game DataFrames.

In [47]:
import pandas as pd

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(full_game)
game_df

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
1,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,...,0.893,8,35,43,10,7,3,16,16,-18.0
